# Export top of the book & trades using dedicated service - Python

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual tradres among market activity. Please refer to <i><a href="">bestex-pointintime</a></i> and <i><a href="">bestex-interval</a></i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run Top of the book & Trades extraction sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2 as tick_trades_and_book
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2_grpc as tick_trades_and_book_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick raw request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 3, day = 5), 
    end_date = date.Date(year = 2021, month = 3, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 15, minutes = 0, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

The following code snippets create the market data request and instantiate the service:

In [ ]:
# generate the tick trades and book request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades_and_book.TickTradesAndBookRequest(
    identifiers = [identifier],
    constraints = constraints
)

#### 3.4 Export data
Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [ ]:
import csv
from datetime import timedelta

filename = 'normalized_ticks_{0}-{1}.csv'.format(ticker, exchange)

with open(filename, mode='w') as normalized_ticks_file:
    normalized_ticks_writer = csv.writer(normalized_ticks_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row, header line
    normalized_ticks_writer.writerow(['Timestamp', 'Bid price', 'Bid price', 'Ask size', 'Ask price', 'Trade price', 'Trade size'])
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:

            # instantiate the tick trades and book service
            service = tick_trades_and_book_service.TickTradesAndBookServiceStub(channel)

            limits_count, trades_count = 0, 0

            for current in service.TickTradesAndBook(request = request, metadata = [('authorization', token)]):
                # get book limits and set trade fields to nan
                if current.data.time_stamp.seconds > 0:
                    time = datetime.fromtimestamp(current.data.time_stamp.seconds) + timedelta(microseconds = current.data.time_stamp.nanos/1000)
                    bid_price = current.data.book.bid.price
                    bid_size = current.data.book.bid.size
                    ask_price = current.data.book.ask.price
                    ask_size = current.data.book.ask.size
                    trade_price = current.data.trade.price
                    trade_size = current.data.trade.size
                    normalized_ticks_writer.writerow([time, bid_price, bid_size, ask_price, ask_size, trade_price, trade_size])
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())